In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def proc_23me(tsv_filename):
    """Used to quickly parse custom 23 and Me formatted files
    
    File is tab-delimited that is specially formatted from 
    23andMe raw data. The four columns that exist in the data
    are:
        1. CHR: Which contig the SNP is on
        2. POS: Which base pair position the SNP was observed
        3. dbSNP_ID: A unique id for known SNPS
        4. ALLELE: the resulting SNP allele
    
    Args:
        tsv_filename (str): /path/to/special/23andMe
    
    Returns:
        (pd.DataFrame): formatted DataFrame
    """
    # Because of the funky format, custom header is needed
    cols = 'CHR POS dbSNP_ID ALLELE'.split()
    return pd.read_csv(tsv_filename, sep = '\t', names = cols)

In [3]:
def proc_PharmGKB(PharmGKB_filename):
    """Utility function for parsing dbSNP data from PharmGKB.
    
    File is tab-delimited and the first column is uniquely identifying
    id for each annotation. Thus, it is used as the resultant 
    DataFrame's index column
    
    Args:
        PharmGKB_filename (str): /path/to/PharmGKB.tsv
    
    Returns:
        (pd.DataFrame): formatted DataFrame
    """
    return pd.read_csv(PharmGKB_filename, sep = '\t', header=0)

In [4]:
def fix_missing_quotes(filename, prefix = 'fixed_'):
    """One-off function to correct unbalanced quotes in files
    by just removing all quotes (as they aren't needed anyways).
    
    Args:
        filename (str): /path/to/filename
        prefix (str): what the fixed filename should start with (default: 'fixed_')
    
    Returns:
        (str): name of fixed file
    """
    dirname, basename = os.path.split(os.path.abspath(filename))
    new_file = os.path.join(dirname, prefix+basename)
    with open(new_file, 'w') as fixed:
        for line in open(filename):
            line = line.strip()\
                .replace('\"', '')
            print(line, file = fixed)
    return new_file

In [5]:
# Ingest the data
pgkb = proc_PharmGKB(fix_missing_quotes('var_drug_ann.tsv'))
t3am = proc_23me('23andme_v5_hg19_ref.txt')

In [6]:
# Filter out pertinent items before the merge
predicate = np.logical_and(
    pgkb['Phenotype Category'] == 'efficacy', 
    pgkb['Significance'] == 'yes'
)
pgkb_w_predicate = pgkb[predicate]

In [7]:
# What I want my finished product columns to be
merge_cols = 'Annotation_ID dbSNP_ID GENE_SYMBOL DRUG_NAME PMID PHENOTYPE_CATEGORY SIGNIFICANCE NOTES SENTENCE ALLELE_PharmGKB ALLELE_23andme'.split()

In [8]:
# Handle the custom data cleaning here
# because PharmGKB has different column names
# compared to 23andMe. This could have been
# handled in the ingest step, but I always like
# keeping the data as close to stock for as long
# as possible.
merged = pd.DataFrame(
    pgkb_w_predicate.merge(t3am, 
        left_on='Variant', 
        right_on = 'dbSNP_ID')\
    .loc[:,[
        'Annotation ID',
        'dbSNP_ID', 
        'Gene',
        'Chemical', 
        'PMID', 
        'Phenotype Category', 
        'Significance', 
        'Notes', 
        'Sentence', 
        'Alleles', 
        'ALLELE']]
)

# Finalize the "look" of the DataFrame
merged.columns = merge_cols

In [9]:
print(f'Number of intersected SNPS: {merged.shape[0]}')

Number of intersected SNPS: 1157


In [10]:
# Uncomment to write to file (if that's your thing)
# merged.drop('Annotation_ID', 1).to_csv('23andme_PharmGKB_map.tsv', sep = '\t')

---
# Extra Credit

In [11]:
# Don't mess up a good thing
ec_merged = merged.copy(deep=True)

## Sanity check
If any entry in this dataset has a comman (,) in the
`DRUG_NAME` and `GENE_SYMBOL` columns, this means they will 
become multiple entries tied to the same `Annotation_ID`. 

Therefore, I want to identify some of those rows in the data.

In [12]:
# Generate the boolean mask
multi_genes = ec_merged.GENE_SYMBOL.str.contains(',')
mutli_drugs = ec_merged.DRUG_NAME.str.contains(',')

In [13]:
san_check = ec_merged[(multi_genes) & (mutli_drugs)].sample(1).squeeze()

In [14]:
annot_id = san_check.Annotation_ID
print(f'Annotation ID: {annot_id}')
genes = san_check.GENE_SYMBOL.split(',')
len_genes = len(genes)
print(f'Gene Symbols (n = {len_genes}):')
print('\t' + '\n\t'.join(genes))
drugs = san_check.DRUG_NAME.split(',')
len_drugs = len(drugs)
print(f'Drug Names (n = {len_drugs}):')
print('\t'+ '\n\t'.join(drugs))

Annotation ID: 981501391
Gene Symbols (n = 2):
	IFNL3 (PA134952671)
	IFNL4 (PA166049147)
Drug Names (n = 3):
	peginterferon alfa-2a (PA164749390)
	peginterferon alfa-2b (PA164784024)
	ribavirin (PA451241)


Now, to turn our head on the sanity check, these drug/gene combos
may affect multiple annotations. So, we want to
quickly find out how many `dbSNP_ID`s are associated with this drug/gene
combo.

In [15]:
# Get the boolean masks ready
goi = genes[0]
doi = drugs[0]
gene_found = ec_merged.GENE_SYMBOL.str.contains(goi, regex=False)
drug_found = ec_merged.DRUG_NAME.str.contains(doi, regex=False)

# Look at the data
combos = ec_merged[(gene_found) & (drug_found)]
print(f'Gene of interest:       {goi}')
print(f'Drug of interest:       {doi}')
print(f'Number of associations: {combos.shape[0]}')
display(combos)

Gene of interest:       IFNL3 (PA134952671)
Drug of interest:       peginterferon alfa-2a (PA164749390)
Number of associations: 60


,Annotation_ID,dbSNP_ID,GENE_SYMBOL,DRUG_NAME,PMID,PHENOTYPE_CATEGORY,SIGNIFICANCE,NOTES,SENTENCE,ALLELE_PharmGKB,ALLELE_23andme
234,769248374,rs12980275,IFNL3 (PA134952671),"peginterferon alfa-2a (PA164749390),peginterfe...",19749757,efficacy,yes,This variant is associated with null virologic...,Allele A is associated with increased response...,A,A
235,1444843603,rs12980275,IFNL3 (PA134952671),"peginterferon alfa-2a (PA164749390),peginterfe...",25769643,efficacy,yes,especially in HCV genotype 1/4 patients.,Genotype AA is associated with increased respo...,AA,A
237,1444705811,rs12980275,IFNL3 (PA134952671),"peginterferon alfa-2a (PA164749390),ribavirin ...",21987611,efficacy,yes,This genotype is associated with sustained vir...,Genotype AA is associated with increased respo...,AA,A
238,1449164615,rs12980275,IFNL3 (PA134952671),"peginterferon alfa-2a (PA164749390),peginterfe...",29369421,efficacy,yes,The AA genotype of rs12980275 was found signif...,Genotype AA is associated with increased respo...,AA,A
239,769248378,rs8099917,IFNL3 (PA134952671),"peginterferon alfa-2a (PA164749390),peginterfe...",19749757,efficacy,yes,This variant is associated with null virologic...,Allele T is associated with increased response...,T,T
241,981479632,rs8099917,IFNL3 (PA134952671),"peginterferon alfa-2a (PA164749390),peginterfe...",21503910,efficacy,yes,Treatment was with ribavirin plus peginterfero...,Genotype TT is associated with increased respo...,TT,T
242,981479639,rs8099917,IFNL3 (PA134952671),"peginterferon alfa-2a (PA164749390),peginterfe...",21503910,efficacy,yes,Treatment was with ribavirin plus peginterfero...,Genotypes GG + GT are associated with increase...,GG + GT,T
243,981481554,rs8099917,IFNL3 (PA134952671),"peginterferon alfa-2a (PA164749390),ribavirin ...",21321200,efficacy,yes,no GG patients were seen. SVR (at 24 weeks) wa...,Genotype TT is associated with increased respo...,TT,T
244,981481567,rs8099917,IFNL3 (PA134952671),"peginterferon alfa-2a (PA164749390),peginterfe...",21254157,efficacy,yes,These patients had HCV genotype 2. TT patients...,Genotype TT is associated with increased respo...,TT,T
245,981483529,rs8099917,IFNL3 (PA134952671),"peginterferon alfa-2a (PA164749390),peginterfe...",21360545,efficacy,yes,Patients had HCV genotypes 2a and 2b. Genotype...,Genotype TT is associated with increased respo...,TT,T


## With this information, we should expect some larger numbers than expected

In [16]:
# Separate multiple entries by converting them to a list
sanity_free = ec_merged.copy(deep = True)
sanity_free.GENE_SYMBOL = sanity_free.GENE_SYMBOL.str.split(',')
sanity_free.DRUG_NAME = sanity_free.DRUG_NAME.str.split(',')

Take those newly created lists and `explode` them into individual
rows, like what we saw in the SQL homework.

In [17]:
clean_explosion = sanity_free.explode('DRUG_NAME')\
    .reset_index(drop=True)\
    .explode('GENE_SYMBOL')\
    .reset_index(drop=True)

In [18]:
# Isolate our conditions with single item columns
check_goi = clean_explosion.GENE_SYMBOL.str.contains(goi, regex=False)
check_doi = clean_explosion.DRUG_NAME.str.contains(doi, regex=False)
explode_check = clean_explosion[(check_goi) & (check_doi)]
display(explode_check)
print(f'Number of associations: {explode_check.shape[0]}')

,Annotation_ID,dbSNP_ID,GENE_SYMBOL,DRUG_NAME,PMID,PHENOTYPE_CATEGORY,SIGNIFICANCE,NOTES,SENTENCE,ALLELE_PharmGKB,ALLELE_23andme
338,769248374,rs12980275,IFNL3 (PA134952671),peginterferon alfa-2a (PA164749390),19749757,efficacy,yes,This variant is associated with null virologic...,Allele A is associated with increased response...,A,A
341,1444843603,rs12980275,IFNL3 (PA134952671),peginterferon alfa-2a (PA164749390),25769643,efficacy,yes,especially in HCV genotype 1/4 patients.,Genotype AA is associated with increased respo...,AA,A
346,1444705811,rs12980275,IFNL3 (PA134952671),peginterferon alfa-2a (PA164749390),21987611,efficacy,yes,This genotype is associated with sustained vir...,Genotype AA is associated with increased respo...,AA,A
348,1449164615,rs12980275,IFNL3 (PA134952671),peginterferon alfa-2a (PA164749390),29369421,efficacy,yes,The AA genotype of rs12980275 was found signif...,Genotype AA is associated with increased respo...,AA,A
350,769248378,rs8099917,IFNL3 (PA134952671),peginterferon alfa-2a (PA164749390),19749757,efficacy,yes,This variant is associated with null virologic...,Allele T is associated with increased response...,T,T
355,981479632,rs8099917,IFNL3 (PA134952671),peginterferon alfa-2a (PA164749390),21503910,efficacy,yes,Treatment was with ribavirin plus peginterfero...,Genotype TT is associated with increased respo...,TT,T
358,981479639,rs8099917,IFNL3 (PA134952671),peginterferon alfa-2a (PA164749390),21503910,efficacy,yes,Treatment was with ribavirin plus peginterfero...,Genotypes GG + GT are associated with increase...,GG + GT,T
361,981481554,rs8099917,IFNL3 (PA134952671),peginterferon alfa-2a (PA164749390),21321200,efficacy,yes,no GG patients were seen. SVR (at 24 weeks) wa...,Genotype TT is associated with increased respo...,TT,T
363,981481567,rs8099917,IFNL3 (PA134952671),peginterferon alfa-2a (PA164749390),21254157,efficacy,yes,These patients had HCV genotype 2. TT patients...,Genotype TT is associated with increased respo...,TT,T
366,981483529,rs8099917,IFNL3 (PA134952671),peginterferon alfa-2a (PA164749390),21360545,efficacy,yes,Patients had HCV genotypes 2a and 2b. Genotype...,Genotype TT is associated with increased respo...,TT,T


Number of associations: 60


## Final part: Grouping and aggregation

In [19]:
cols_of_interest = ['GENE_SYMBOL', 'DRUG_NAME', 'dbSNP_ID']
final_ec = clean_explosion.loc[:,cols_of_interest].groupby(['DRUG_NAME', 'GENE_SYMBOL'], as_index = False).aggregate(';'.join)

In [20]:
# Uncomment to write to file (if that's your thing)
# final_ec.to_csv('ec_dbSNPS.tsv', sep = '\t')

With all of our sanity checks, we should end up with a single entry that summarizes
all the conditions shown above instead of a multiple entries. This means that every
gene/drug combo seen in this dataset should have one and only one entry in this 
summarized DataFrame.

In [21]:
final_goi = final_ec.GENE_SYMBOL == goi
final_doi = final_ec.DRUG_NAME == doi
final_check = final_ec[(final_goi) & (final_doi)]
print(f'Gene of interest:        {goi}')
print(f'Drug of interest:        {doi}')
print(f'Total associated dbSNPs: {final_check.dbSNP_ID.str.count(";").values[0] + 1}')
display(final_check)

Gene of interest:        IFNL3 (PA134952671)
Drug of interest:        peginterferon alfa-2a (PA164749390)
Total associated dbSNPs: 60


,DRUG_NAME,GENE_SYMBOL,dbSNP_ID
710,peginterferon alfa-2a (PA164749390),IFNL3 (PA134952671),rs12980275;rs12980275;rs12980275;rs12980275;rs...
